## **Домашняя работа 2. Гриневская Катя, БКЛ182**

Сначала импортирую всё необходимое

In [1]:
# ! pip install pymorphy2
from pymorphy2 import MorphAnalyzer

In [2]:
morph = MorphAnalyzer()

In [65]:
# устанавливаем mystem
# ! pip install pymystem3
from pymystem3 import Mystem
m = Mystem()

In [4]:
# !pip install natasha

In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [6]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [7]:
# ! pip install slovnet

In [8]:
# !pip install sklearn

In [9]:
from sklearn.metrics import accuracy_score

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [12]:
! pip install flair

  Using cached https://files.pythonhosted.org/packages/cd/19/902d1691c1963ab8c9a9578abc2d65c63aa1ecf4f8200143b5ef91ace6f5/flair-0.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/01/47358efec5396fc80f98273c42cbdfe7aab056252b07884ffcc0f118978f/konoha-4.6.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/36/9e022b76a3ac440e1d750c64fa6152469f988efe0c568b945e396e2693b5/pytest-6.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/50/21/92c3cfe56f5c0647145c4b0083d0733dd4890a057eb100a8eeddf949ffe9/gdown-3.12.2.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached https://files.pythonhosted.org/packages/76/a1/05d7f62f956d77b23a640efc650f80

  ERROR: Could not find a version that satisfies the requirement torch>=1.1.0 (from flair) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch>=1.1.0 (from flair)


In [13]:
import re

In [14]:
# !pip install spacy

In [15]:
# !python -m spacy download en_core_web_sm

In [16]:
import en_core_web_sm
nlp = en_core_web_sm.load()

## Русский язык

В тексте написанном мной, содержатся следующие затруднительные для POS-теггинга моменты:
1) аббревиатура
2) солва с дефисом (предлоги, существительные, прилагательные)
3) имена собственные
4) сложные предлоги
5) сленговые слова

### Импортируем текст и теги и токенизируем через nltk

In [17]:
with open('hw_2_rus.txt', encoding='utf-8') as fh:
    text = fh.read()

In [18]:
with open('hw_2_rus_tagged.txt', encoding='utf-8') as fh:
    text_tagged = fh.read()

In [19]:
tokens = word_tokenize(text)

In [20]:
pos_tags_commas = word_tokenize(text_tagged)

### 1) pymorphy2

In [21]:
pymorph_pos = []
for token in tokens:
    if token not in """!@#$%^&*()_+'?/.<>,|\"``''~№=-:;""":
        anal = morph.parse(token)
        pymorph_pos.append(anal[0].tag.POS)

In [22]:
len(pymorph_pos)

96

In [23]:
my_pos = []
for tag in pos_tags_commas:
    if tag not in """!@#$%^&*()_+'?/.<>,|\"``''~№=-:;""":
        my_pos.append(tag)

In [24]:
len(my_pos)

96

Количество тэгов совпадает с количеством токенов (проверила на всякий случай) - можно идти дальше

Теперь мне нужно привести тэги к единому формату - я буду приводить к своему

In [25]:
pymorph_stand_pos = []
for tag in pymorph_pos:
    if tag in 'NOUN':
        pymorph_stand_pos.append('N')
    elif (tag in 'ADJF') or (tag in 'ADJS') or (tag in 'COMP'):
        pymorph_stand_pos.append('ADJ')
    elif (tag in 'VERB') or (tag in 'INFN'):
        pymorph_stand_pos.append('V')
    elif tag in 'NUMR':
        pymorph_stand_pos.append('NUMB')
    elif (tag in 'PRTF') or (tag in 'PRTS'):
        pymorph_stand_pos.append('PRT')
    elif (tag in 'ADVB') or (tag in 'PRED'):
        pymorph_stand_pos.append('AVD')
    elif tag in 'NPRO':
        pymorph_stand_pos.append('PRON')
    else:
        pymorph_stand_pos.append(tag)

In [26]:
len(pymorph_stand_pos)

96

In [27]:
print('Accuracy: %.4f' % accuracy_score(pymorph_stand_pos, my_pos))

Accuracy: 0.8646


### 2) теперь mystem

In [28]:
mystem_pos = []
for d in m.analyze(text):
    if 'analysis' in d.keys():
        mystem_pos.append(d['analysis'][0]['gr'].split(',')[0].split('=')[0])

In [29]:
mystem_stand_pos = []
for tag in mystem_pos:
    if (tag == 'NUM') or (tag == 'ANUM'):
        mystem_stand_pos.append('NUMB')
    elif tag == 'PR':
        mystem_stand_pos.append('PREP')
    elif tag == 'S':
        mystem_stand_pos.append('N')
    elif (tag == 'SPRO') or (tag == 'APRO') or (tag == 'ADVPRO'):
        mystem_stand_pos.append('PRON')
    elif tag == 'A':
        mystem_stand_pos.append('ADJ')
    elif tag == 'PART':
        mystem_stand_pos.append('PRCL')
    else:
        mystem_stand_pos.append(tag)

In [30]:
print('Accuracy: %.4f' % accuracy_score(mystem_stand_pos, my_pos))

Accuracy: 0.8958


### 3) Natasha

In [31]:
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [32]:
natasha_pos = []
for obj in doc.tokens:
    if obj.pos != 'PUNCT':
        natasha_pos.append(obj.pos)

In [33]:
len(natasha_pos)

96

Количество тэгов совпадает => идём дальше

In [34]:
natasha_stand_pos = []
for tag in natasha_pos:
    if (tag == 'NOUN') or (tag == 'PROPN'):
        natasha_stand_pos.append('N')
    elif tag == 'ADP':
        natasha_stand_pos.append('PREP')
    elif tag == 'NUM':
        natasha_stand_pos.append('NUMB')
    elif tag == 'VERB':
        natasha_stand_pos.append('V')
    elif tag.endswith('CONJ'):
        natasha_stand_pos.append('CONJ')
    elif tag == 'DET':
        natasha_stand_pos.append('PRON')
    else:
        natasha_stand_pos.append(tag)

In [35]:
print('Accuracy: %.4f' % accuracy_score(natasha_stand_pos, my_pos))

Accuracy: 0.8750


## Английский язык

В тексте написанном мной, содержатся следующие затруднительные для POS-теггинга моменты:
1) слова с дефисом
2) сленг/обсценная лексика
3) имена собственные
4) одинаковые словоформы для разных частей речи

### Импортируем текст и теги и токенизируем через nltk

In [36]:
with open('hw_2_eng.txt', encoding='utf-8') as fh:
    text = fh.read()

In [37]:
with open('hw_2_eng_tagged.txt', encoding='utf-8') as fh:
    text_tagged = fh.read()

In [38]:
tokens_dirty = word_tokenize(text)

In [39]:
tokens = []
for i in range(len(tokens_dirty)):
    if tokens_dirty[i] not in """!@#$%^&*()_+'?/.<>,|\"``''~№=-:;""":
        tokens.append(tokens_dirty[i])

In [40]:
len(tokens)

116

In [41]:
pos_tags_commas = word_tokenize(text_tagged)

In [42]:
my_pos = []
for tag in pos_tags_commas:
    if tag not in """!@#$%^&*()_+'?/.<>,|\"``''~№=-:;""":
        my_pos.append(tag)

In [43]:
len(my_pos)

116

### 1) NLTK

In [44]:
nltk_pos = []
for tag in nltk.pos_tag(tokens, tagset='universal'):
    if tag[1] not in """!@#$%^&*()_+'?/.<>,|\"``''~№=-:;""":
        nltk_pos.append(tag[1])

In [45]:
len(nltk_pos)

116

Всё сошлось, ура! Стандартизируем

In [46]:
nltk_stand_pos = []
for tag in nltk_pos:
    if tag == 'VERB':
        nltk_stand_pos.append('V')
    elif tag == 'NOUN':
        nltk_stand_pos.append('N')
    elif (tag == 'ADP') or (tag == 'PRT'):
        nltk_stand_pos.append('PREP')
    elif tag == 'NUM':
        nltk_stand_pos.append('NUMB')
    else:
        nltk_stand_pos.append(tag)

Считаем accuracy

In [47]:
print('Accuracy: %.4f' % accuracy_score(nltk_stand_pos, my_pos))

Accuracy: 0.8793


### 2) SpaCy

SpaCy токенизирует слова с дефисом как 2 слова, поэтому тут добавляю специальное правило

In [48]:
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

In [49]:
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # EDIT: commented out regex that splits on hyphens between letters:
        #r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

In [50]:
infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer
doc = nlp(text)

In [51]:
doc = nlp(text)

In [52]:
spacy_pos = []
for i, s in enumerate(doc.sents):
    for t in s:
        if (t.pos_ != 'PUNCT') and (t.pos_ != 'SPACE'):
            print(t.text, ' ', t.pos_)
            spacy_pos.append(t.pos_)

Get   AUX
the   DET
hell   NOUN
out   SCONJ
of   ADP
here   ADV
said   VERB
Jenny   PROPN
to   ADP
her   DET
well-paid   ADJ
colleague   NOUN
on   ADP
Wednesday   PROPN
Fuck   VERB
off   ADP
answered   VERB
Peter   PROPN
You   PRON
'll   VERB
never   ADV
know   VERB
how   ADV
much   ADJ
pain   NOUN
I   PRON
feel   VERB
about   ADP
it   PRON
You   PRON
are   AUX
psycho   ADJ
All   DET
that   DET
glisters   NOUN
is   AUX
not   PART
gold   ADJ
This   DET
job   NOUN
will   VERB
put   VERB
gold   NOUN
right   ADV
in   ADP
my   DET
pocket   NOUN
while   SCONJ
you   PRON
think   VERB
it   PRON
's   AUX
shit   NOUN
Boss   NOUN
said   VERB
I   PRON
can   VERB
increase   VERB
my   DET
income   NOUN
twice   ADV
I   PRON
am   AUX
not   PART
going   VERB
to   PART
pay   VERB
attention   NOUN
to   ADP
this   DET
Well   INTJ
I   PRON
am   AUX
not   PART
afraid   ADJ
I   PRON
have   AUX
spent   VERB
eight   NUM
hard-working   ADJ
years   NOUN
here   ADV
and   CCONJ
I   PRON
know   VERB
I   PRON
am   A

In [53]:
len(spacy_pos)

116

Всё сошлось, ура! Стандартизируем.

In [54]:
spacy_stand_pos = []
for tag in spacy_pos:
    if (tag == 'AUX') or (tag == 'VERB'):
        spacy_stand_pos.append('V')
    elif (tag == 'PROPN') or (tag == 'NOUN'):
        spacy_stand_pos.append('N')
    elif tag.endswith('CONJ'):
        spacy_stand_pos.append('CONJ')
    elif tag == 'NUM':
        spacy_stand_pos.append('NUMB')
    elif (tag == 'ADP') or (tag == 'PART'):
        spacy_stand_pos.append('PREP')
    else:
        spacy_stand_pos.append(tag)

Считаем accuracy

In [55]:
print('Accuracy: %.4f' % accuracy_score(spacy_stand_pos, my_pos))

Accuracy: 0.8534


### 3) Flair

In [56]:
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter


# initialize sentence splitter
splitter = SegtokSentenceSplitter()

# use splitter to split text into list of sentences
sentences = splitter.split(text)

# predict tags for sentences
tagger = SequenceTagger.load('upos')
tagger.predict(sentences)

# iterate through sentences and print predicted labels
flair_pos = []
for sentence in sentences:
    tagged_sent = sentence.to_tagged_string()
    for tag in re.findall(r'<\w+>', tagged_sent):
      if tag != '<PUNCT>':
        flair_pos.append(tag.replace('<', '',).replace('>', ''))

SyntaxError: EOL while scanning string literal (<ipython-input-56-b115eb0cf751>, line 12)

In [ ]:
len(flair_tags)

Всё сошлось, ура! Стандартизируем.

In [ ]:
flair_stand_pos = []
for tag in flair_pos:
    if tag == 'VERB':
        flair_stand_pos.append('V')
    elif tag == 'NOUN':
        flair_stand_pos.append('N')
    elif (tag == 'ADP') or (tag == 'PRT'):
        flair_stand_pos.append('PREP')
    elif tag == 'NUM':
        flair_stand_pos.append('NUMB')
    else:
        flair_stand_pos.append(tag)

Считаем accuracy

In [ ]:
print('Accuracy: %.4f' % accuracy_score(flair_stand_pos, my_pos))

## Возвращаемся к русскому языку. Берём лучший теггер для русского языка: у меня это mystem. Выделим 3 вида синтаксических групп: 1) не + прилагательное/наречие 2) не + глагол + __; 3) прил+сущ. Выделим такие n-граммы.

Взяла именно такие группы, потому что рассчитываю на частые словосочетания в отзывах: 
1. _не интересный, не впечатляющий, не приятный, не захватывающий_ в отрицательных отзывах;
2. _не показался интересным/весёлым/..., не понравился фильм/сюжет/состав/..., не привлёк внимание, не уважаю такое_ чтобы знать, что после глагола;
3. _плохой/отстойный/ужасный фильм_ и _хороший/потрясный/захватывающий фильм_ не будут сливаться в одно

In [94]:
def ngrams(otz):
    ne_ad = [] # здесь будут не+прилагательные
    ne_verb_sm = [] # здесь не+глагол+что-то
    adj_n = [] # здесь прил+сущ
    a = m.analyze(otz.lower())
    for i in range(len(a)):
        if 'analysis' in a[i].keys():
            if (a[i]['text'] == 'не') and (a[i+2]['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'A'):
                ne_ad.append('не'+' '+a[i+2]['text'])
            elif (a[i]['text'] == 'не') and (a[i+2]['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V'):
                if (len(a)-i) > 4:
                    ne_verb_sm.append('не' + ' ' + a[i+2]['text'] + ' ' +  a[i+4]['text'])
                else:
                    ne_verb_sm.append('не' + ' ' + a[i+2]['text'])
            elif ((a[i]['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'A') and
                  (a[i+2]['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'S')):
                if (len(a)-i) > 2:
                    adj_n.append(a[i]['text'] + ' ' + a[i+2]['text'])
                else:
                    continue
    return ne_ad, ne_verb_sm, adj_n

Проверяем

In [95]:
otz = """этот парень вообще не красивый и совсем Не умный и я не люблю кошек и грозных собак"""
ne_ad, ne_verb_sm, adj_n = ngrams(otz)

In [100]:
ne_ad

['не красивый', 'не умный']

In [101]:
ne_verb_sm

['не люблю кошек']

In [102]:
adj_n

['грозных собак']